In [1]:
import string
import nltk
# EDIT
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction import text

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit, when
from pyspark.sql.types import StringType
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [5]:
def clean_text (text) :
    words_List = nltk.word_tokenize(text)
    final_list = [elto for elto in words_List if elto not in STOP_WORDS]
    return " ".join(final_list)

In [6]:
def preprocess(df):
    # Include the username in the text to increase classification accuracy
    # From tests we can see that adding it before or after processing the text data doesn't matter
    df.loc[:, 'message'] = df['message'] + ' ' + df['username']

    # Notice that we want Sleep = SLEEP = SlEEp = sleeP ETC   
    df.loc[:, 'message'] = df.loc[:, 'message'].str.lower()

    # Drop NaN values
    df.dropna(inplace=True, subset=['channel', 'message'])

    # Remove words like: can, could, will, been, would...
    df.loc[:, 'message'] = df.loc[:, 'message'].apply(clean_text)

    # stem separate words
    stemmer = SnowballStemmer("english")
    df.loc[:, 'message'] = df.loc[:, 'message'].astype(str).str.split()
    df.loc[:, 'message'] = df.loc[:, 'message'].apply(lambda x: [stemmer.stem(word) for word in x])

    # Remove rows with empty messages
    df = df[df['message'].astype(bool)]

    # Rejoin list of messages to single string message separated by <space>
    df.loc[:, 'message'] = df.loc[:, 'message'].apply(lambda x: ' '.join(x))

    df.rename(columns={'channel': 'label'}, inplace=True)

    final_df = df.loc[:, ['message', 'label']]

    # print(final_df)

    return final_df

In [7]:
MODELS_PATH = 'models\\'
MODEL = 'multinomialNB'
STOP = text.ENGLISH_STOP_WORDS
STOP_WORDS = list(STOP) + list(string.punctuation)

# Create a list of predictions to concat all predictions and later save them into .csv format
# for further processing
predictions_list = []

mapping = {0:'#loltyler1', 1:'#gothamchess'}

globals()['models_loaded'] = False
globals()['my_model'] = None

globals()['my_model'] = NaiveBayesModel.load(MODELS_PATH+MODEL)
globals()['models_loaded'] = True

In [8]:
# globals()['models_loaded'] = False
# globals()['my_model'] = None

# Toy predict function. Normally you'd use your loaded globals()['my_model'] here
# def predict(df):
#     df.show()
#     print(globals()['my_model'])
#     predictions = globals()['my_model'].transform(df)
#     # predictions = predictions.withColumn('prediction', 
#     #                     when(col('prediction') == 0, lit(mapping[0])).otherwise(lit(mapping[1])))
    
#     predictions.show()
#     return predictions.prediction

# predict_udf = udf(predict, StringType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to PySpark DataFrame
    df = spark.read.json(rdd)
    
    # Convert to Pandas DataFrame for preprocessing
    df_pandas = df.toPandas()
    df_pandas = preprocess(df_pandas)
    # Reconvert to PySpark DataFrame (I'm sure there is a better way to do this)
    df = spark.createDataFrame(df_pandas)

    # break the sentence into a list of words
    tokenizer = Tokenizer(inputCol="message", outputCol="words")
    words_data = tokenizer.transform(df)

    # TF section
    hashing_TF = HashingTF(inputCol='words', outputCol='rawFeatures', numFeatures=200000)
    featurized_data = hashing_TF.transform(words_data)

    # IDF section
    idf = IDF(inputCol='rawFeatures', outputCol='features')
    idf_model = idf.fit(featurized_data)

    rescaled_data = idf_model.transform(featurized_data)
    
    # rescaled_data.show()

    # # Utilize our predict function
    # df_withpreds = df.withColumn("pred", predict_udf(
    #     struct([df[x] for x in df.columns])
    # ))
    # df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = NaiveBayesModel.load(MODELS_PATH+MODEL)
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    df_result = globals()['my_model'].transform(rescaled_data)
    df_result = df_result.withColumn('prediction', 
        when(col('prediction') == 0, lit(mapping[0])).otherwise(lit(mapping[1])))


    to_concat = df_result.toPandas()
    predictions_list.append(to_concat)

    try:
        df_result.select(['message', 'label', 'probability', 'prediction']).show()
    except Exception as e:
        pass
        # Uncomment to see what went wrong
        # print(e)

In [9]:
print(globals()['my_model'])

NaiveBayesModel: uid=NaiveBayes_b32a0b916ba2, modelType=multinomial, numClasses=2, numFeatures=200000


In [10]:
ssc = StreamingContext(sc, 10)

In [11]:
lines = ssc.socketTextStream("localhost", 8080)
lines.foreachRDD(process)

In [12]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2022-05-23 19:22:20 =========
Success
+--------------------+----------+--------------------+------------+
|             message|     label|         probability|  prediction|
+--------------------+----------+--------------------+------------+
|   kekw lol darwangg|#loltyler1|[0.41396012296278...|#gothamchess|
|  kekw toddygotbeatz|#loltyler1|[0.99999819794434...|  #loltyler1|
|   kek snowmanofpoop|#loltyler1|[0.90440650882311...|  #loltyler1|
|jebait jebait jeb...|#loltyler1|[0.57274347345359...|  #loltyler1|
|jebait cherry_buiiet|#loltyler1|[0.54130153870317...|  #loltyler1|
|  rank ianskilamang1|#loltyler1|[0.99999993893297...|  #loltyler1|
|        kekw cap2567|#loltyler1|[0.99999984103421...|  #loltyler1|
|      kekw raptusrit|#loltyler1|[0.73050045815373...|  #loltyler1|
|      kekw nyekon_11|#loltyler1|[0.73050045815373...|  #loltyler1|
|ianskilamang1 acc...|#loltyler1|[1.0,1.8046667086...|  #loltyler1|
|    jebait tobyraven|#loltyler1|[0.99996263729261...|  #loltyler1|


In [13]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


In [43]:
# pd.concat(predictions_list, ignore_index=True)